In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from tensorflow.keras import layers

# Load your cleaned dataset
df = pd.read_csv("Fully_cleaned.csv")

# Clean and preprocess
df['bed'] = pd.to_numeric(df['bed'], errors='coerce')
df['bath'] = df['bath'].replace(r'\+', '', regex=True)
df['bath'] = pd.to_numeric(df['bath'], errors='coerce')
df = df.dropna(subset=['bed', 'bath', 'Safety_Score'])

# Target transformation
df['Property Value_log'] = np.log1p(df['Property Value'])

# Features and target split
X = df.drop(columns=['Property Value', 'Property Value_log'])
y = df['Property Value_log']

# One-hot encode categoricals
X = pd.get_dummies(X, drop_first=True)

# Replace inf/-inf, fill NaNs
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# DNN Model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32,
                    validation_split=0.1, verbose=1)

# Predict and evaluate
y_pred_log = model.predict(X_test_scaled).flatten()
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"✅ RMSE: {rmse:,.2f}")
print(f"✅ R² Score: {r2:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(y_true, y_pred, alpha=0.4)
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("DNN: Actual vs Predicted Housing Prices")
plt.grid(True)
plt.show()